In [1]:
!pip install sklearn yellowbrick

In [2]:
import sys
sys.path.append('../utils/')

In [3]:
from utils import CustomTokenizer, CustomPreprocessor

In [4]:
import pandas as pd
import numpy as np
from collections import OrderedDict
import boto3

In [5]:
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
import nltk
from nltk.corpus import stopwords

nltk.download(["punkt", "stopwords", "wordnet"])

[nltk_data] Downloading package punkt to /home/hadoop/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/hadoop/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/hadoop/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
from yellowbrick.text import TSNEVisualizer

In [7]:
s3 = boto3.resource('s3',  region_name='us-east-1')
# obj = s3.Object(bucket_name='st1800newsdataset', key='dataset/news.csv')
obj = s3.Object(bucket_name='newsaws', key='news.csv')
data_frame = pd.read_csv(obj.get()['Body'])

In [8]:
data_frame.head()

,id,id_news,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [9]:
data_frame['content'].describe()

count            142570
unique           142038
top       advertisement
freq                 42
Name: content, dtype: object

In [10]:
news_data = data_frame['title'] + ' ' + data_frame['content'] + ' ' + data_frame['author'] + ' ' + data_frame['publication']

In [11]:
news_data.dropna()
news_data.count()

126693

In [12]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=CustomTokenizer(), stop_words='english')

In [ ]:
news_tfidf = tfidf_vectorizer.fit_transform(news_data.astype(str))

/home/hadoop/env/local/lib64/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_word

In [ ]:
tr_svd = TruncatedSVD(n_components=250, random_state=42)

In [ ]:
news_svd = tr_svd.fit_transform(news_tfidf)

In [ ]:
tr_svd.explained_variance_ratio_.sum()

In [ ]:
clusters = KMeans(n_clusters=5)
clusters.fit(news_svd)

In [ ]:
tsne = TSNEVisualizer()
tsne.fit(news_svd, ["Cluster {}".format(c) for c in clusters.labels_])
tsne.poof()

In [ ]:
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(news_svd, clusters.labels_, sample_size=1000))